<a href="https://colab.research.google.com/github/saurabh-parkar/Sentiment_Analysis/blob/master/Sentiment_Analysis_TextCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import time
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets
from tqdm import tqdm
import matplotlib.pyplot as plt

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
TEXT = data.Field(lower=True, batch_first=True, include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [3]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:12<00:00, 6.85MB/s]


In [4]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                          
100%|█████████▉| 399222/400000 [00:21<00:00, 18512.54it/s]

In [6]:

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [ ]:
for i in train_iterator:
  print(min ([ min(i.text[1]) for i in train_iterator]))

tensor(10, device='cuda:0')


In [36]:
class TextCNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, output_dim, dropout, pad_index):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_index)
    
    self.convolutions = []
    self.filter_num_width = [(64, 3), (64, 4), (64, 5)]

    for out_channel, filter_width in self.filter_num_width:
      self.convolutions.append(
          nn.Conv2d(
              1,           # in_channel
              out_channel, # out_channel
              kernel_size=(embedding_dim, filter_width), # (height, width)
              bias=True
              )
      )

    self.lin_input_dim = sum([x for x, y in self.filter_num_width])

    self.dropout=nn.Dropout()
    self.batch_norm = nn.BatchNorm1d(self.lin_input_dim, affine=False)

    self.out = nn.Linear((self.lin_input_dim), output_dim)


  def forward(self, text, text_lengths):
    # text = [batch size,sent_length]
    x = self.embedding(text)
    # embedded = [batch size, sent_len, emb dim]

    x = torch.transpose(x.view(x.size()[0], 1, x.size()[1], -1), 2, 3)

    x = self.conv_layers(x)

    x = self.dropout(x)

    x = self.out(x)

    return x


  def conv_layers(self, x):
    chosen_list = list()
    for conv in self.convolutions:
      feature_map = F.tanh(conv(x))
      # (batch_size, out_channel, 1, max_word_len-width+1)
      chosen = torch.max(feature_map, 3)[0]
      # (batch_size, out_channel, 1)            
      chosen = chosen.squeeze()
      # (batch_size, out_channel)
      chosen_list.append(chosen)
    
    # (batch_size, total_num_filers)
    return torch.cat(chosen_list, 1)


In [18]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM1 = 256
HIDDEN_DIM2 = 100
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = TextCNN(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, DROPOUT, PAD_IDX)
model

TextCNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (dropout): Dropout(p=0.5, inplace=False)
  (batch_norm): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (out): Linear(in_features=192, out_features=1, bias=True)
)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model)} trainable paramters')

The model has 2500393 trainable paramters


In [25]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [26]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4645, -2.2566,  1.2649,  ..., -0.9130, -0.3423,  0.0470],
        [-1.4452, -0.6232, -0.5869,  ..., -0.8689,  0.5159,  1.6829],
        [ 0.4926,  0.9945,  0.9569,  ...,  0.6560,  0.4407, -0.5550]])

In [37]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4645, -2.2566,  1.2649,  ..., -0.9130, -0.3423,  0.0470],
        [-1.4452, -0.6232, -0.5869,  ..., -0.8689,  0.5159,  1.6829],
        [ 0.4926,  0.9945,  0.9569,  ...,  0.6560,  0.4407, -0.5550]])


In [43]:
optimizer = optim.Adam(model.parameters())


In [44]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [45]:

def accuracy(preds, y):
    predicted = torch.round(torch.sigmoid(preds))
    correct = (predicted == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [46]:
def train(model, iterator, optimizer, criterion):

    model.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        optimizer.zero_grad()
        # retrieve text and no. of words
        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label.squeeze())

        acc = accuracy(predictions, batch.label)

        # perform backpropagation
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [47]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label.squeeze())

            acc = accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [48]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [49]:
N_EPOCHS = 10

def run_train(epochs, model, train_iterator, valid_iterator, optimizer, criterion, model_type):
    best_valid_loss = float('inf')

    for epoch in range(epochs):

        start_time = time.time()
      
        # train the model
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

        # evaluate the model
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)


        # save the best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'saved_weights'+'_'+model_type+'.pt')

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

In [50]:
run_train(N_EPOCHS, model, train_iterator, valid_iterator, optimizer, criterion, "BiLSTM")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 01 | Epoch Time: 5m 15s
	Train Loss: 0.709 | Train Acc: 50.44%
	 Val. Loss: 0.690 |  Val. Acc: 56.53%
Epoch: 02 | Epoch Time: 5m 18s
	Train Loss: 0.696 | Train Acc: 52.06%
	 Val. Loss: 0.689 |  Val. Acc: 53.10%
Epoch: 03 | Epoch Time: 5m 19s
	Train Loss: 0.689 | Train Acc: 53.97%
	 Val. Loss: 0.685 |  Val. Acc: 54.74%
Epoch: 04 | Epoch Time: 5m 20s
	Train Loss: 0.680 | Train Acc: 56.51%
	 Val. Loss: 0.680 |  Val. Acc: 57.04%
Epoch: 05 | Epoch Time: 5m 19s
	Train Loss: 0.671 | Train Acc: 58.61%
	 Val. Loss: 0.669 |  Val. Acc: 63.35%
Epoch: 06 | Epoch Time: 5m 20s
	Train Loss: 0.652 | Train Acc: 61.68%
	 Val. Loss: 0.653 |  Val. Acc: 65.68%
Epoch: 07 | Epoch Time: 5m 18s
	Train Loss: 0.627 | Train Acc: 64.93%
	 Val. Loss: 0.630 |  Val. Acc: 71.03%
Epoch: 08 | Epoch Time: 5m 20s
	Train Loss: 0.599 | Train Acc: 68.26%
	 Val. Loss: 0.607 |  Val. Acc: 71.22%
Epoch: 09 | Epoch Time: 5m 21s
	Train Loss: 0.565 | Train Acc: 71.57%
	 Val. Loss: 0.584 |  Val. Acc: 72.67%
Epoch: 10 | Epoch T

In [51]:
# Total 15 Epochs

N_EPOCHS = 5
run_train(N_EPOCHS, model, train_iterator, valid_iterator, optimizer, criterion, "TextCNN")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 01 | Epoch Time: 5m 19s
	Train Loss: 0.497 | Train Acc: 76.64%
	 Val. Loss: 0.540 |  Val. Acc: 75.17%
Epoch: 02 | Epoch Time: 5m 18s
	Train Loss: 0.466 | Train Acc: 78.73%
	 Val. Loss: 0.526 |  Val. Acc: 75.11%
Epoch: 03 | Epoch Time: 5m 17s
	Train Loss: 0.442 | Train Acc: 80.04%
	 Val. Loss: 0.520 |  Val. Acc: 74.41%
Epoch: 04 | Epoch Time: 5m 20s
	Train Loss: 0.418 | Train Acc: 81.78%
	 Val. Loss: 0.503 |  Val. Acc: 75.83%
Epoch: 05 | Epoch Time: 5m 21s
	Train Loss: 0.391 | Train Acc: 83.28%
	 Val. Loss: 0.490 |  Val. Acc: 76.87%


In [53]:
# Total 20 Epochs

N_EPOCHS = 5
run_train(N_EPOCHS, model, train_iterator, valid_iterator, optimizer, criterion, "TextCNN")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 01 | Epoch Time: 5m 18s
	Train Loss: 0.362 | Train Acc: 84.83%
	 Val. Loss: 0.480 |  Val. Acc: 77.03%
Epoch: 02 | Epoch Time: 5m 21s
	Train Loss: 0.346 | Train Acc: 85.72%
	 Val. Loss: 0.476 |  Val. Acc: 76.91%
Epoch: 03 | Epoch Time: 5m 19s
	Train Loss: 0.322 | Train Acc: 87.08%
	 Val. Loss: 0.471 |  Val. Acc: 77.09%
Epoch: 04 | Epoch Time: 5m 20s
	Train Loss: 0.306 | Train Acc: 87.81%
	 Val. Loss: 0.476 |  Val. Acc: 76.18%
Epoch: 05 | Epoch Time: 5m 21s
	Train Loss: 0.281 | Train Acc: 89.01%
	 Val. Loss: 0.461 |  Val. Acc: 77.86%


In [54]:
# Testing

model.load_state_dict(torch.load('/content/saved_weights_TextCNN.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Test Loss: 0.479 | Test Acc: 77.02%
